In [1]:
!pip install transformers==4.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 16.1 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.2/184.2 kB 16.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 56.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [47 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-cpython-310
      creating build/lib.linux-x86_64-cpython-310/tokenizers
      copying py_src/tokenizers/__init__.py -> build/lib.linux-x86_64-cpython-310/tokenizers
      creating build/lib.linux-x86_64-cpython-310/tokenizers/models
      copying py_src/tokenizers/models/__init__.py

In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
import math
import os
from tempfile import TemporaryDirectory
from typing import Tuple
import numpy as np
import logging
import json
import pandas as pd

import torch
from torch import nn,Tensor
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
import matplotlib.pyplot as plt

from tqdm import tqdm
from PIL import Image

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [4]:
real_path='/kaggle/input/socofing/SOCOFing/Real'
easy_path='/kaggle/input/socofing/SOCOFing/Altered/Altered-Easy'
mid_path='/kaggle/input/socofing/SOCOFing/Altered/Altered-Medium'
hard_path='/kaggle/input/socofing/SOCOFing/Altered/Altered-Hard'

In [5]:
def fingeridmaker(file_path):
    
    if file_path.endswith('Left_thumb_finger.BMP'):
        return 0
    elif file_path.endswith('Left_index_finger.BMP'):
        return 1
    elif file_path.endswith('Left_middle_finger.BMP'):
        return 2
    elif file_path.endswith('Left_ring_finger.BMP'):
        return 3
    elif file_path.endswith('Left_little_finger.BMP'):
        return 4
    elif file_path.endswith('Right_thumb_finger.BMP'):
        return 5
    elif file_path.endswith('Right_index_finger.BMP'):
        return 6
    elif file_path.endswith('Right_middle_finger.BMP'):
        return 7
    elif file_path.endswith('Right_ring_finger.BMP'):
        return 8
    elif file_path.endswith('Right_little_finger.BMP'):
        return 9

In [6]:
#combined_dataset=[]

In [7]:
import os
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

# Assuming each image has dimensions (width, height)
image_width = 96
image_height = 96


class CustomDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.transform = transforms.Compose([
            transforms.Resize((image_height, image_width)),
            transforms.ToTensor(),
            transforms.Normalize(mean=0.5, std=0.5)
        ])
        self.data = []  # List to store image data and labels
        self.file_list = os.listdir(root_dir)

        # Collect image paths and labels
        '''
        for person_id in os.listdir(self.root_dir):
            person_dir = os.path.join(self.root_dir, person_id)
            if os.path.isdir(person_dir):
                for finger_id in os.listdir(person_dir):
                    finger_dir = os.path.join(person_dir, finger_id)
                    if os.path.isdir(finger_dir):
                        for variation_id, image_file in enumerate(os.listdir(finger_dir)):
                            image_path = os.path.join(finger_dir, image_file)
                            self.data.append((image_path, int(person_id), int(finger_id), variation_id))
        '''
        for i in range(1, 601):
          bmp_files = [f for f in self.file_list if f.endswith('.BMP') and f.startswith(str(i))]
          for path in bmp_files:
                #print(path)
                #if path=='171__M_Left_little_finger.BMP':
                  #print(path)
                person_id=str(i-1)
                finger_id=str(fingeridmaker(path))
                path=os.path.join(root_dir, path)
                #print(path)
                self.data.append((path, person_id, finger_id))
                
                
            
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        #image_path, person_id, finger_id, variation_id = self.data[idx]
        image_path, person_id, finger_id=self.data[idx]
        #print(image_path)

        if image_path is not None:
          image = Image.open(image_path).convert("L")  # Convert to grayscale
          image = self.transform(image)          
          #combined_dataset.append((image, person_id, finger_id))
          #print(image, person_od)        
          #print(combined_dataset)
          return image, person_id, finger_id
        else:
            pass

In [8]:
batch_size = 16
real_dataset=CustomDataset(real_path)
easy_dataset=CustomDataset(easy_path)
hard_dataset=CustomDataset(hard_path)
mid_dataset=CustomDataset(mid_path)


real_dataloader = DataLoader(real_dataset, batch_size=batch_size, shuffle=False)
easy_dataloader = DataLoader(easy_dataset, batch_size=batch_size, shuffle=False)
mid_dataloader = DataLoader(mid_dataset, batch_size=batch_size, shuffle=False)
hard_dataloader = DataLoader(hard_dataset, batch_size=batch_size, shuffle=False)

In [9]:
from torch.utils.data import ConcatDataset
combined_dataset = ConcatDataset([real_dataset, easy_dataset, hard_dataset, mid_dataset])


In [10]:
print(len(combined_dataset))

155844


In [11]:
#print(zip(real_dataloader, easy_dataloader, hard_dataloader, mid_dataloader))

'''
for real_batch in zip(real_dataloader, easy_dataloader, hard_dataloader, mid_dataloader):
  print(real_batch, easy_batch, hard_batch, mid_batch)
  final_images=ConcatDataset([real_batch[0]])
  final_person_ids=ConcatDataset([real_batch[1], easy_batch[1], mid_batch[1], hard_batch[1]])
  final_finger_ids=ConcatDataset([real_batch[2], easy_batch[2], mid_batch[2], hard_batch[2]])
  combined_dataset.append((final_images, final_person_ids, final_finger_ids))
  
for idx, real_batch in enumerate(real_dataloader):  
  combined_dataset.append((real_batch[0], real_batch[1], real_batch[2]))
print('real complete')
    
for idx,easy_batch in enumerate(easy_dataloader): 
  combined_dataset.append((easy_batch[0], easy_batch[1], easy_batch[2]))
print('easy complete')

for idx, mid_batch in enumerate(mid_dataloader):  
  combined_dataset.append((mid_batch[0], mid_batch[1], mid_batch[2]))
print('mid complete')
    
for idx, hard_batch in enumerate(hard_dataloader):  
  combined_dataset.append((hard_batch[0], hard_batch[1], hard_batch[2]))
print('hard complete')
'''
                        

"\nfor real_batch in zip(real_dataloader, easy_dataloader, hard_dataloader, mid_dataloader):\n  print(real_batch, easy_batch, hard_batch, mid_batch)\n  final_images=ConcatDataset([real_batch[0]])\n  final_person_ids=ConcatDataset([real_batch[1], easy_batch[1], mid_batch[1], hard_batch[1]])\n  final_finger_ids=ConcatDataset([real_batch[2], easy_batch[2], mid_batch[2], hard_batch[2]])\n  combined_dataset.append((final_images, final_person_ids, final_finger_ids))\n  \nfor idx, real_batch in enumerate(real_dataloader):  \n  combined_dataset.append((real_batch[0], real_batch[1], real_batch[2]))\nprint('real complete')\n    \nfor idx,easy_batch in enumerate(easy_dataloader): \n  combined_dataset.append((easy_batch[0], easy_batch[1], easy_batch[2]))\nprint('easy complete')\n\nfor idx, mid_batch in enumerate(mid_dataloader):  \n  combined_dataset.append((mid_batch[0], mid_batch[1], mid_batch[2]))\nprint('mid complete')\n    \nfor idx, hard_batch in enumerate(hard_dataloader):  \n  combined_dat

In [12]:
def dataset_split(dataset, ratio):
    train_size = int(ratio * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
    return train_dataset, val_dataset

In [13]:
batch_size=16
final_dataset = combined_dataset
train_dataset, val_dataset=dataset_split(final_dataset, 0.8)
train_dataset, test_dataset=dataset_split(train_dataset, 0.9)
train_dataloader=DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader=DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_dataloader=DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [14]:
import torch.nn as nn

In [184]:
import torch.nn as nn

class ResidualBlock(nn.Module):
    def __init__(self,  in_channels, out_channels, strides=1, use_1conv=True):
        super().__init__()
        self.use_1conv = use_1conv
        
        self.model1 = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=in_channels, kernel_size=3, stride=1, padding=1),
            nn.LazyBatchNorm2d(),
            nn.ReLU(),
            nn.Conv2d(in_channels=in_channels, out_channels=in_channels, kernel_size=3, padding=1),
            nn.LazyBatchNorm2d()
        )
        self.model2 = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=in_channels, kernel_size=3, stride=strides, padding=1),
            nn.LazyBatchNorm2d(),
            nn.ReLU(),
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, padding=1),
            nn.LazyBatchNorm2d()
        )
        self.conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=1, stride=strides, padding=0)
    
    def forward(self, x):
        y = self.model1(x)

        #print(y.shape, x.shape)
        y+=x
        nn.ReLU()(y)
        y=self.model2(x)
        if self.use_1conv:
           x = self.conv(x)
        #print(x.shape, y.shape)
        y += x
        return nn.ReLU()(y)

In [185]:
class FingerprintClassifier(nn.Module):
    def __init__(self, output_class, use_1conv=True):
        super().__init__()
        self.model=nn.Sequential(                      
            nn.LazyConv2d(out_channels=64, kernel_size=(7, 7), stride=2, padding=3),
            nn.LazyBatchNorm2d(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            ResidualBlock(in_channels=64, out_channels=64, use_1conv=True),
            ResidualBlock(in_channels=64, out_channels=128, strides=2, use_1conv=True),
            ResidualBlock(in_channels=128, out_channels=256, use_1conv=True),
            ResidualBlock(in_channels=256, out_channels=512, strides=2, use_1conv=True),
            nn.AdaptiveAvgPool2d((1, 1)), 
            nn.Flatten(),
            nn.LazyLinear(output_class)
        )        
    def forward(self, x):
        x=self.model(x)
        return x

In [192]:
learning_rate=0.01
model=FingerprintClassifier(600)
#optimizer = optim.SGD(model.parameters(), lr=learning_rate,\
                     #momentum=0.9, nesterov=True)
optimizer = AdamW(model.parameters(), lr=learning_rate, correct_bias=False)

/opt/conda/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [200]:
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt

print_every=100

device = 'cuda' if torch.cuda.is_available() else 'cpu'
def train_fingerprintmodel_(model, optimizer, epochs=2): 
    model = model.to(device=device)  # Move the model parameters to CPU/GPU

    # Lists to store training and validation errors for plotting
    training_errors = []
    validation_errors = []    

    for e in range(epochs):
        for idx, batch_data in enumerate(train_dataloader):
            model.train()  # Put model to training mode
            images = batch_data[0]
            persons_id = batch_data[1]
            finger_id = batch_data[2]
            #print(type(images), type(persons_id), type(finger_id))
            # Ensure that images, persons_id, and finger_id are tensors
            #images = torch.stack(images)
            images = torch.stack([torch.tensor(img) for img in images])
            persons_id = [int(pid) for pid in persons_id]
            persons_id = torch.tensor(persons_id)
            #finger_id = torch.tensor(finger_id)
            
            
            images = images.to(device=device)  # Move to device, e.g., GPU
            
            persons_id = persons_id.to(device=device)
            #finger_id=finger_id.to(device=device)
            #print(images.shape)
            scores = model(images)
            print(scores, len(scores), len(scores[0]), persons_id)
            #print(len(scores), print(persons_id))
            loss = F.cross_entropy(scores, persons_id)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backward pass: compute the gradient of the loss with
            # respect to each parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backward pass.
            optimizer.step()

            if idx % print_every == 0:
                # Compute training error
                model.eval()  # Put model to evaluation mode
                with torch.no_grad():
                    y_pred = model(images)
                    training_error = F.cross_entropy(y_pred, persons_id).item()
                    training_errors.append(training_error)

                # Compute validation error
                val_error = 0.0
                num_batches = 0
                num_correct=0
                num_samples=0
                model.eval()  # Put model to evaluation mode
                with torch.no_grad():
                    for x_val, y_val, _ in val_dataloader:
                        x_val=torch.tensor(x_val)
                        y_val = [int(pid) for pid in y_val]
                        y_val=torch.tensor(y_val)
                        x_val = x_val.to(device=device)                        
                        y_val = y_val.to(device=device)
                        y_pred_val = model(x_val)
                        val_error += F.cross_entropy(y_pred_val, y_val).item()
                        _, preds = y_pred_val.max(1)
                        print(preds, y_val)
                        num_correct += (preds == y_val).sum()
                        num_samples += preds.size(0)                       
                        
                        num_batches += 1
                val_error /= num_batches
                validation_errors.append(val_error)
                accuracy=(num_correct/num_samples)
                
                

                print(f'Epoch {e}, Iteration {idx}, Training Loss = {training_error:.4f}, Validation Loss = {val_error:.4f}, Val_accuracy = {accuracy:.4f}')

    # Plot the training and validation errors
    plt.figure()
    plt.plot(training_errors, label='Training Error')
    plt.plot(validation_errors, label='Validation Error')
    plt.xlabel('Iterations')
    plt.ylabel('Cross Entropy Loss')
    plt.title('Training and Validation Errors')
    plt.legend()
    plt.show()

In [ ]:
'''train_fingerprintmodel까지가 모델에서 쓰이는 부분입니다. 아래 test도 잘 작동해요. \
나머지 아랫부분은 그냥.. 혹시 다시 쓰일까봐 남겨둔 애들? classification으로 접근을 하니, gradient가 쎈 애들만 학습해버리는건지, 아님 그냥 모델이 이상한건지, resnet18이 
전혀 먹히질 않습니다.. 다 1 아님 4로 분류하네요'''

In [ ]:
train_fingerprintmodel_(model, optimizer, epochs=2)

/tmp/ipykernel_28/114884504.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  images = torch.stack([torch.tensor(img) for img in images])
/tmp/ipykernel_28/114884504.py:68: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_val=torch.tensor(x_val)


tensor([[ 1.7280,  1.8135,  1.7405,  ..., -3.3991, -3.2930, -4.2863],
        [ 1.7178,  1.8139,  1.7884,  ..., -2.4586, -2.9081, -2.9715],
        [ 1.7562,  1.8806,  1.8048,  ..., -2.9220, -3.0327, -3.4181],
        ...,
        [ 1.7522,  1.8691,  1.7899,  ..., -3.0798, -3.1084, -3.6731],
        [ 1.6597,  1.6989,  1.7234,  ..., -2.3852, -2.9899, -3.2149],
        [ 1.6634,  1.7062,  1.7276,  ..., -2.3881, -2.9839, -3.1967]],
       device='cuda:0', grad_fn=<AddmmBackward0>) 16 600 tensor([ 38,   4, 563,  25,   2, 257, 344,  56,  23,   2,   4,   2,  66,   2,
         18, 583], device='cuda:0')
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 4, 1], device='cuda:0') tensor([ 51,   0,  33,   2,  42,   0, 299, 328,  39, 328, 438,   0,  45,  12,
         58,   2], device='cuda:0')
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1], device='cuda:0') tensor([  0,  23,  47,   9,  20, 105,  21, 181,   9,   4, 242,   3,  24,  45,
          2,  51], device='cuda:0')
tensor([4, 1, 1, 1,

In [ ]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)  # move to device, e.g. GPU
            y_val = [int(pid) for pid in y_val]
            y_val = y_val.to(device=device)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [ ]:
state_dict = model.state_dict()
model_path = os.path.join('/kaggle/working/', f"kobert_epoch_1.pth")
#logger.info(f"global_step: {global_step} model saved at {model_path}")
torch.save(state_dict, model_path)

In [ ]:
def inference(model, test_dataloader):
    pred=[]
    model.eval()
    aggregate=0
    correct=0
    for batch_data in test_dataloader:
        with torch.no_grad():
            images = batch_data[0]
            persons_id = batch_data[1]
            finger_id = batch_data[2]            
            images = torch.stack([torch.tensor(img) for img in images])
            persons_id = [int(pid) for pid in persons_id]
            persons_id = torch.tensor(persons_id)   
            images = images.to(device=device)  # Move to device, e.g., GPU           
            persons_id = persons_id.to(device=device)
            
            score=model(images)
            _, prediction=torch.max(score, 1)
            
            aggregate += persons_id.size(0)
            correct += (prediction == persons_id).sum().item()
            
            pred.append((prediction, persons_id))
            
    accuracy = correct / aggregate
    return pred, accuracy


In [ ]:
pred, acc=inference(model, test_dataloader)
print(acc)

In [ ]:
pred=inference(model, test_dataloader)
def accuracy(pred)

In [ ]:
import numpy as np
import os

# Get the working directory path in Kaggle
working_dir = '/kaggle/working/'

# Iterate through the final dataloader to get batches of combined data
for batch in final_dataloader:
    combined_images, combined_person_ids, combined_finger_ids = batch
    
    # Iterate through the images in the batch
    for i in range(combined_images.size(0)):
        image = combined_images[i]
        person_id = combined_person_ids[i]
        finger_id = combined_finger_ids[i]
        
        # Reshape image tensor to (C, H*W)
        c, h, w = image.size()
        image_reshaped = image.view(c, -1)
        
        # Convert image tensor to numpy array
        image_np = image_reshaped.numpy()
        
        # Save as CSV in /kaggle/working/
        csv_filename = os.path.join(working_dir, f"combined_person_{person_id}_finger_{finger_id}.csv")
        np.savetxt(csv_filename, image_np, delimiter=",")


In [ ]:
image_path='/kaggle/input/socofing/SOCOFing/Real/171__M_Left_little_finger.BMP'
pic=Image.open(image_path)

pic


In [ ]:
##old_reader
def bmp_reader(file_path):   #returns bmp files read in dimensions of label
  real_path = file_path

# List all files in the folder
  file_list = os.listdir(real_path)
  real=[]

# Filter only BMP image files
  for i in range(1, 601):
    bmp_files = [f for f in file_list if f.endswith('.BMP') and f.startswith(str(i))]
    label_images=[]
    for bmp_file in bmp_files:
      image_path = os.path.join(real_path, bmp_file)
      image = Image.open(image_path)
      label_images.append(image)
    real.append(label_images)
  
  return real
    

real=bmp_reader(real_path)
easy=bmp_reader(easy_path)
mid=bmp_reader(mid_path)
hard=bmp_reader(hard_path)

print(len(real), len(easy), len(mid))


#DO NOT RERUN THIS CELL. IT WILL READ THE ENTIRE DATASET AGAIN. CALL THE VARIABLE

In [ ]:
def combined(real_path, easy_path, hard_path, ):   #returns bmp files read in dimensions of label
  real_path = file_path

# List all files in the folder
  file_list = os.listdir(real_path)
  real=[]

# Filter only BMP image files
  for i in range(1, 601):
    bmp_files = [f for f in file_list if f.endswith('.BMP') and f.startswith(str(i))]
    label_images=[]
    for bmp_file in bmp_files:
      image_path = os.path.join(real_path, bmp_file)
      image = Image.open(image_path)
      label_images.append(image)
    real.append(label_images)
  
  return real
    

In [ ]:
def Dataset(bmp_list):    
    bmp_list=torch.tensor(bmp_list)
    bmp_list.reshape(600, 20, 96, 103)    
    return bmp_list, bmp_list.shape
        
        
    

In [ ]:
batch_size = 16
dataloader = DataLoader(final_dataset batch_size=batch_size, shuffle=True)

# Example usage of the DataLoader
for batch in dataloader:
    images, person_ids = batch

In [ ]:
# Create the dataset

batch_size = 16
real_dataset=CustomDataset(real_path)
easy_dataset=CustomDataset(easy_path)
hard_dataset=CustomDataset(hard_path)
mid_dataset=CustomDataset(mid_path)

real_dataloader = DataLoader(real_dataset, batch_size=batch_size, shuffle=True)
easy_dataloader= DataLoader(easy_dataset, batch_size=batch_size, shuffle=True)
mid_dataloader= DataLoader(mid_dataset, batch_size=batch_size, shuffle=True)
hard_dataloader= DataLoader(hard_dataset, batch_size=batch_size, shuffle=True)

combined_samples = []
for real_batch, easy_batch, hard_batch, mid_batch in zip(real_dataloader, easy_dataloader, hard_dataloader, mid_dataloader):
    combined_images = torch.cat((real_batch[0], easy_batch[0], hard_batch[0], mid_batch[0]), dim=1)  # Concatenate the images along the second dimension
    combined_person_labels = torch.cat((real_batch[1], easy_batch[1], hard_batch[1], mid_batch[1]))  # Combine person labels
    #combined_finger_labels = torch.cat((real_batch[2], easy_batch[2], hard_batch[2], mid_batch[2]))  # Combine finger labels
    combined_samples.append((combined_images, combined_person_labels))

# Create a new ConcatDataset from the combined samples
final_dataset = ConcatDataset(combined_samples)
final_dataloader=DataLoader(final_dataset, batch_size=batch_size, shuffle=True)


#final_dataset=ConcatDataset([real_dataloader, easy_dataloader], dim=-2)

In [ ]:


# Create a DataLoader for batch processing

    # Now you can process the batch of images along with their labels

In [ ]:
image_path='/kaggle/input/socofing/SOCOFing/Real/1__M_Left_index_finger.BMP'
pic=Image.open(image_path)

pic
